In [25]:
import pandas as pd
import numpy as np
from glob import glob
import cv2
import matplotlib.pylab as plt

In [26]:
glioma_tumour_files = glob(r'C:\Users\denni\OneDrive\Desktop\brain-tumour-project\Brain-Tumor-Classification-DataSet\Training\glioma_tumor/*.jpg')
meningioma_tumor_files = glob(r'C:\Users\denni\OneDrive\Desktop\brain-tumour-project\Brain-Tumor-Classification-DataSet\Training\meningioma_tumor/*.jpg')
no_tumour_files = glob(r'C:\Users\denni\OneDrive\Desktop\brain-tumour-project\Brain-Tumor-Classification-DataSet\Training\no_tumor/*.jpg')
pituitary_tumor_file = glob(r'C:\Users\denni\OneDrive\Desktop\brain-tumour-project\Brain-Tumor-Classification-DataSet\Training\pituitary_tumor/*.jpg')

In [27]:
def reading_images(link_list):
    lst = []
    for image_idx in range(len(link_list)):
        image_array = plt.imread(link_list[image_idx])
        lst.append(image_array)
    return lst

glioma_images_lst = reading_images(glioma_tumour_files)
menigioma_images_lst = reading_images(meningioma_tumor_files)
no_tumour_images_lst = reading_images(no_tumour_files)
pituitary_images_lst = reading_images(pituitary_tumor_file)

In [28]:
print("Number of glioma tumours are",len(glioma_images_lst))
print("Number of menigioma tumours are",len(menigioma_images_lst))
print("Number of no tumours are",len(no_tumour_images_lst))
print("Number of pituitary tumours are",len(pituitary_images_lst))

Number of glioma tumours are 826
Number of menigioma tumours are 822
Number of no tumours are 395
Number of pituitary tumours are 827


In [29]:
classes =['glioma-tumour', 'menigioma-tumour', 'no-tumour', 'pituitary-tumour']
true_labels = []
true_labels += [0]*len(glioma_images_lst)
true_labels += [1]*len(menigioma_images_lst)
true_labels += [2]*len(no_tumour_images_lst)
true_labels += [3]*len(pituitary_images_lst)

In [30]:
all_images_lst = glioma_images_lst + menigioma_images_lst + no_tumour_images_lst + pituitary_images_lst

In [31]:
unique_lst=[]
for i in range(len(all_images_lst)):
    if all_images_lst[i].shape not in unique_lst: 
        unique_lst.append(all_images_lst[i].shape)
print("The number of different shapes are",len(unique_lst))

The number of different shapes are 361


In [32]:
def resize_images(image_list, target_size):
    for i in range(len(image_list)):
        image = image_list[i]
        image_list[i] = cv2.resize(image, target_size)

target_size = (256, 256)  # Set your desired dimensions

resize_images(all_images_lst, target_size)

In [33]:
unique_lst=[]
for i in range(len(all_images_lst)):
    if all_images_lst[i].shape not in unique_lst: 
        unique_lst.append(all_images_lst[i].shape)
print("The number of different shapes are",len(unique_lst))

The number of different shapes are 1


In [39]:
all_images_array = np.array(all_images_lst)

In [34]:
true_labels = [[value] for value in true_labels]
true_labels = np.array(true_labels, dtype=np.uint8)
true_labels = true_labels.reshape(-1,)

In [41]:
# creating split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_images_array, true_labels, test_size=0.3, random_state=42)

In [43]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models

In [44]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [47]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10


32/32 [==============================] - 78s 2s/step - loss: 426.0528 - accuracy: 0.4520 - val_loss: 0.8515 - val_accuracy: 0.6818
Epoch 2/10
32/32 [==============================] - 74s 2s/step - loss: 0.5476 - accuracy: 0.8133 - val_loss: 0.5770 - val_accuracy: 0.7933
Epoch 3/10
32/32 [==============================] - 75s 2s/step - loss: 0.2145 - accuracy: 0.9253 - val_loss: 0.5103 - val_accuracy: 0.8537
Epoch 4/10
32/32 [==============================] - 74s 2s/step - loss: 0.0532 - accuracy: 0.9841 - val_loss: 0.6532 - val_accuracy: 0.8769
Epoch 5/10
32/32 [==============================] - 75s 2s/step - loss: 0.0152 - accuracy: 0.9960 - val_loss: 0.5892 - val_accuracy: 0.8955
Epoch 6/10
32/32 [==============================] - 74s 2s/step - loss: 0.0063 - accuracy: 0.9990 - val_loss: 0.7144 - val_accuracy: 0.8897
Epoch 7/10
32/32 [==============================] - 74s 2s/step - loss: 0.0036 - accuracy: 0.9990 - val_loss: 0.7628 - val_accuracy: 0.8966
Epoch 8/10
32/32

In [50]:
model.evaluate(X_test,y_test)

27/27 [==============================] - 5s 168ms/step - loss: 0.6903 - accuracy: 0.8862


[0.69031822681427, 0.8861788511276245]

In [51]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = model.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

27/27 [==============================] - 5s 170ms/step
Classification Report: 
               precision    recall  f1-score   support

           0       0.93      0.93      0.93       259
           1       0.81      0.84      0.83       252
           2       0.85      0.70      0.77       114
           3       0.93      0.98      0.95       236

    accuracy                           0.89       861
   macro avg       0.88      0.86      0.87       861
weighted avg       0.89      0.89      0.88       861

